In [ ]:
"""
TimeSeries_RNN_20230604.py
David Nilsson - Prime Fitness Studio AB
2023-06-04
"""

In [ ]:
# Import needed libraries
import tensorflow as tf
print('TensorFlow version:', tf.__version__)

In [ ]:
# from tensorflow import keras
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils  import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt

print('Keras version:',tf.keras.__version__)

In [ ]:
# Matlab plotting
import matplotlib
import matplotlib.pyplot as plt
#mpl.rcParams['figure.figsize'] = (8, 6)
#mpl.rcParams['axes.grid'] = False

In [ ]:
# Helper libraries
import os
import time
import datetime

In [ ]:
import numpy as np
import pandas as pd
import sklearn
from   sklearn.model_selection import train_test_split

In [ ]:
import IPython
import IPython.display
import seaborn as sns
import sys

In [ ]:
"""
To easier optimize the hyperparameters the function build_model() could be used.

"""
# Defining a Keras model to search optimized hyper parameters
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten())
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 3)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )
    if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(10, activation="softmax"))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [ ]:
"""
Functions to be called
"""
# Help function to get indices for columns based on column names
def get_label_columns_indices(dataframe,label_columns=None):
  if label_columns is not None:
    label_columns_indices = {name: i for i, name in enumerate(label_columns)}
  else:
    label_columns_indices = {name: i for i, name in enumerate(dataframe.columns)}
  return label_columns_indices

In [ ]:
# Here is our function that take our time sequence data and convert it to a dataset 
# that then can be us as a dataset for training or evaluation
# Note that this code only implements a target/label width of one, as it seems that timeseries_dataset_from_array only allows for that
def datasetgen(dataframe, input_width=24, label_width=1, shift=1, batch_size=8,
               label_columns=None, start_index=None, end_index=None, shuffle=False):
  offset = input_width+shift-1       # offset to where targets start (and input ends) // Fixed 2023-05-01
  input_data = dataframe[:-offset]   # Input up until where the target starts
  label_columns_indices = get_label_columns_indices(dataframe,label_columns) # get the selected columns
  targets = dataframe[list(label_columns_indices)]
  targets = targets[offset:]         # Output from end of input and for the selected columns
  # Here we create the windows and store them as a dataset using 'timeseries_dataset_from_array'
  train_ds = tf.keras.utils.timeseries_dataset_from_array(
    input_data, targets, sequence_length=input_width,
    sequence_stride=1, sampling_rate=1, batch_size=batch_size, shuffle=shuffle,
    start_index=start_index, end_index=end_index)
  return train_ds

In [ ]:
MAX_EPOCHS = 30

In [ ]:
# Function to compile the base model and a callback to stop if no progress
# https://keras.io/api/callbacks/early_stopping/
def compile_and_fit(model, patience=5):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min',
                                                    restore_best_weights = True)

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])

  start = time.time()
  history = model.fit(train_ds, epochs=MAX_EPOCHS,
                      validation_data=val_ds,
                      callbacks=[early_stopping])
  end = time.time()
  print("Time to run:", end - start)

  return history,model

In [ ]:
# define a plot function to show how well it is prediction
def show_plot(plot_data, delta, title):
    labels = ["History", "True Future", "Model Prediction"]
    marker = [".-", "rx", "go"]
    time_steps = list(range(-(plot_data[0].shape[0]), 0))
    if delta:
        future = delta
    else:
        future = 0

    plt.title(title)
    for i, val in enumerate(plot_data):
        if i:
            plt.plot(future, plot_data[i], marker[i], markersize=10, label=labels[i])
        else:
            plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
    plt.legend()
    plt.xlim([time_steps[0], (future + 5) * 2])
    plt.xlabel("Time-Step")
    plt.show()
    return

In [ ]:
# Comparing the trained model against the base_line-model
def last_time_step_mse(Y_true, Y_pred):
   return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

In [ ]:
# Test for GPU and determine what GPU we have
gpu_devices = tf.config.list_physical_devices('GPU')

In [ ]:
if not gpu_devices:
    print("No GPU was detected. Neural nets can be very slow without a GPU.")
    if "google.colab" in sys.modules:
        print("Go to Runtime > Change runtime and select a GPU hardware "
              "accelerator.")
    if "kaggle_secrets" in sys.modules:
        print("Go to Settings > Accelerator and select GPU.")
else:
    print("No !nvidia-smi -L")

In [ ]:
# If possible to run code wiht 16 bits float instad of 32 bits float, this code acitvates such functionality:
if gpu_devices:
 details = tf.config.experimental.get_device_details(gpu_devices[0])
 compute_capability=details.get('compute_capability')
 print("Compute capability:",compute_capability)
 if compute_capability[0]>6:
   print("Turning on mixed_float16")
   policy = tf.keras.mixed_precision.Policy('mixed_float16')
   tf.keras.mixed_precision.set_global_policy(policy)

In [ ]:
zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)
df = pd.read_csv(csv_path)

In [ ]:
# This lab will just deal with hourly predictions, so start by sub-sampling the data from 10 minute intervals to 1h:
df = df[5::6] # starting from index 5 take every 6th record.

In [ ]:
date_time = pd.to_datetime(df.pop('Date Time'), format='%d.%m.%Y %H:%M:%S')

In [ ]:
print(df.shape)
# Let's take a glance at the data. Here are the first few rows:
df.head()

In [ ]:
plot_cols = ['T (degC)', 'p (mbar)', 'rho (g/m**3)']
plot_features = df[plot_cols]
plot_features.index = date_time
_ = plot_features.plot(subplots=True)

In [ ]:
plot_features = df[plot_cols][:480]
plot_features.index = date_time[:480]
_ = plot_features.plot(subplots=True)

In [ ]:
# Printing out the statistics of the loaded data
df.describe().transpose()

In [ ]:
# Pre-processing and cleaning the data from unrealistic values
wv = df['wv (m/s)']
bad_wv = wv == -9999.0
wv[bad_wv] = 0.0

In [ ]:
max_wv = df['max. wv (m/s)']
bad_max_wv = max_wv == -9999.0
max_wv[bad_max_wv] = 0.0

In [ ]:
# The above inplace edits are reflected in the DataFrame
df['wv (m/s)'].min()

In [ ]:
# Processing the wind data to be represented as radians to avoid 
# problems of proximity oc 0 and 360 degrees
wv = df.pop('wv (m/s)')
max_wv = df.pop('max. wv (m/s)')

In [ ]:
# Convert to radians.
wd_rad = df.pop('wd (deg)')*np.pi / 180

In [ ]:
# Calculate the wind x and y components.
df['Wx'] = wv*np.cos(wd_rad)
df['Wy'] = wv*np.sin(wd_rad)

In [ ]:
# Calculate the max wind x and y components.
df['max Wx'] = max_wv*np.cos(wd_rad)
df['max Wy'] = max_wv*np.sin(wd_rad)

In [ ]:
# Plotting out the heatmap of the wind-vector
plt.hist2d(df['Wx'], df['Wy'], bins=(50, 50), vmax=400)
plt.colorbar()
plt.xlabel('Wind X [m/s]')
plt.ylabel('Wind Y [m/s]')
ax = plt.gca()
ax.axis('tight')

In [ ]:
# Converting the Date Time to be representing seconds instead as a string
timestamp_s = date_time.map(datetime.datetime.timestamp)

In [ ]:
# Converting seconds to "Time of Day" and "Time of Year" data
# This is because the temperature is more likely to shift by variations
# over a day and also over a year
day = 24*60*60
year = (365.2425)*day

In [ ]:
df['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
df['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
df['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
df['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))

In [ ]:
# Plotting the converted time-data
plt.plot(np.array(df['Day sin'])[:25])
plt.plot(np.array(df['Day cos'])[:25])
plt.xlabel('Time [h]')
plt.title('Time of day signal')

In [ ]:
# To find the optimal frequences of temperature by time-data
# Fast Fourier Transform can be used
fft = tf.signal.rfft(df['T (degC)'])
f_per_dataset = np.arange(0, len(fft))

In [ ]:
n_samples_h = len(df['T (degC)'])
hours_per_year = 24*365.2524
years_per_dataset = n_samples_h/(hours_per_year)

In [ ]:
f_per_year = f_per_dataset/years_per_dataset
plt.step(f_per_year, np.abs(fft))
plt.xscale('log')
plt.ylim(0, 400000)
plt.xlim([0.1, max(plt.xlim())])
plt.xticks([1, 365.2524], labels=['1/Year', '1/day'])
_ = plt.xlabel('Frequency (log scale)')

In [ ]:
"""
Question 1:
The two parameters that stands out as containing unrealistic or likely faulty/bad values
are minimum wind and minimum value of maximal wind in meter per second. The wind strength can not be -9999 m/s, and 
this is corrected with setting the value to 0.0 m/s if the value for wind is -9999 in the 
dataframe. This is the minimum value of the wind.



Question 2:
The wind direction in degrees is presenting very similar information in wind directions
like 0 to 5 compared to 355 to 360 etc., even though they are very similar values in terms
of real wind direction. Therefore the wind in degrees are converted into radians.



Question 3:
Data of time is only important if it is represented in the right frequencies, wich contains 
shift in the dependent variable (temperature in this case) in days and years. This is because 
the temperature is likely shifting over a day, and is likely to be similar next day at the 
same time. This is also likely to be true for the frequency of year. To find these relevant 
frequencies, a Fast Fourier Transform can be applied to analyse amplitudes of the temperature
relative to the time-data. This data is also represented with sinus and cosinus functions, 
since temperatures are shifting in this form relative time. Then we will have two ways&parameters
to represent relaevant time/frequencies. This is adding up the resolution of the models prediction
capabilities by a sort of "triangulation".

"""

In [ ]:
# Find index to where we want to split our data into our datasets
DataSplitRatios=(0.7,0.2,0.1) # Maybe one should check this sums up to 1?
n = len(df)
split1ix = int(n*DataSplitRatios[0])
split2ix = int(n*(DataSplitRatios[0]+DataSplitRatios[1]))
print(split1ix,split2ix,n)

In [ ]:
# Normalizing the data to be able to use the data to train a model
train_mean = df[0:split1ix].mean()
train_std = df[0:split1ix].std()
dfnorm = (df - train_mean) / train_std

In [ ]:
# Plotting the statistics and distribution as standard deviation
df_std = dfnorm.melt(var_name='Column', value_name='Normalized')
plt.figure(figsize=(12, 6))
ax = sns.violinplot(x='Column', y='Normalized', data=df_std)
_ = ax.set_xticklabels(df.keys(), rotation=90)

In [ ]:
# Test get_label_columns_indices
#lcol= ['T (degC)']
lcol= None
lci = get_label_columns_indices(dfnorm,lcol)
print(lci)

In [ ]:
# Now create the used datasets for part 1
lcol=['T (degC)']
train_ds = datasetgen(dfnorm, input_width=24, label_width=1, shift=1, 
              label_columns=lcol, start_index=0, end_index=split1ix)
val_ds = datasetgen(dfnorm, input_width=24, label_width=1, shift=1, 
              label_columns=lcol, start_index=split1ix, end_index=split2ix)
test_ds = datasetgen(dfnorm, input_width=24, label_width=1, shift=1, 
              label_columns=lcol, start_index=split2ix, end_index=None)

In [ ]:
# Establish test targets (unroll dataset)
test_targets = None
for batch in test_ds:
  inputs, targets = batch
  if test_targets == None:
    test_targets = targets
  else:
   test_targets = tf.concat([test_targets,targets], axis=0)

In [ ]:
# And number of features
num_target_features = targets.shape[1]

In [ ]:
lci = get_label_columns_indices(dfnorm,lcol)

In [ ]:
# Get a batch and look at the shapes for one of the datasets
eval_ds = test_ds
for batch in eval_ds:
  inputs, targets = batch
  batchlen = len(inputs)
  # We here have a batch of seqlen sequences
  print("Input shape = {0}, Output shape = {1}".format(inputs.shape,targets.shape))
  break
# Print dataset size
nobatches = eval_ds.__len__()
print("No sequences =", nobatches.numpy() * batchlen)

In [ ]:
"""
The loss-function in this case, with parametric data and in interval-scale, the 
size of the error is important to take care of, but the greater the error in the predicted
value, the more punished the prediction should be, and this is what Mean Squared Error 
brings us, since the quadratic operations. Also, it would punish equally if the predicted
error is over/under shooting. To measure the predicted value, the absolut value is of relevance, 
hence the Mean Absolute Accuracy should be used to measure the performance of the trained model.

Error: Mean Squared Error
Metric: Mean Absolute Accuracy
"""

In [ ]:
# Define a first simple LSTM model with 128 units
lstm_model_baseline = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(128, recurrent_activation="sigmoid", return_sequences=True),
    tf.keras.layers.LSTM(128, recurrent_activation="sigmoid", return_sequences=True),
    tf.keras.layers.LSTM(128, recurrent_activation="sigmoid", return_sequences=True),
    tf.keras.layers.LSTM(128, recurrent_activation="sigmoid", return_sequences=True),
    tf.keras.layers.LSTM(128, recurrent_activation="sigmoid", return_sequences=True),
    tf.keras.layers.LSTM(128, recurrent_activation="sigmoid", return_sequences=True),
    tf.keras.layers.LSTM(128, recurrent_activation="sigmoid", return_sequences=False),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=num_target_features)
])

In [ ]:
# Compile and train this model
history_lstm_baseline, lstm_model_baseline = compile_and_fit(lstm_model_baseline)

In [ ]:
"""
# Evaluate the model.
test_loss, test_acc = lstm_model_baseline1.evaluate(test_ds, verbose=0)
print('Test accuracy: %.3f' % test_acc)
print('Test loss: %.4f' % test_loss)

# Plot the training curves
pd.DataFrame(history_lstm_baseline.history).plot(figsize=(8,5))
plt.show()



# Plotting the predictions of temperatures
for x, y in val_ds.take(3):
    show_plot(
        [x[0][:, 1].numpy(), y[0].numpy(), lstm_model_baseline1.predict(x)[0]],
        1,
        "Single Step Prediction",
    )
"""

In [ ]:
# Defining a the base model to compare with
base_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='sigmoid'),
    tf.keras.layers.Dense(units=num_target_features)
])

In [ ]:
# Compiling the model
history_base_model, base_model = compile_and_fit(base_model)

In [ ]:
"""
# Comparing the trained model with the baseline model statistically
# https://www.tensorflow.org/api_docs/python/tf/keras/losses/MeanAbsoluteError
base_predictions = base_model.predict(eval_ds)
trained_predictions = lstm_model_baseline.predict(eval_ds)
trained_predictions_gru = better_gru_model_baseline.predict(eval_ds)

base_mae = tf.keras.losses.MeanAbsoluteError(test_targets, base_predictions)
base_mse = tf.keras.losses.MeanAbsoluteError(test_targets, base_predictions)

trained_mae = tf.keras.losses.MeanAbsoluteError(test_targets, trained_predictions)
trained_mse = tf.keras.losses.MeanAbsoluteError(test_targets, trained_predictions)

trained_mae_gru = tf.keras.losses.MeanAbsoluteError(test_targets, trained_predictions_gru)
trained_mse_gru = tf.keras.losses.MeanAbsoluteError(test_targets, trained_predictions_gru)
"""

In [ ]:
"""
# Making the predictions for the test dataset
predictions = model.predict(test_ds)

# Extracting the predictions for the desired time steps (24 and 48 hours ahead)
prediction_24h = predictions[:, 24, lci['T (degC)']]
prediction_48h = predictions[:, 48, lci['T (degC)']]

# Printing the predictions of the temperature
print("Prediction 24 hours ahead:", prediction_24h)
print("Prediction 48 hours ahead:", prediction_48h)
"""

In [ ]:
# Better LSTM model(s): definition, compile and run, evaluate, and compare!
better_lstm_model_baseline = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(128, recurrent_activation="sigmoid", return_sequences=True),
    tf.keras.layers.LSTM(128, recurrent_activation="sigmoid", return_sequences=True),
    tf.keras.layers.LSTM(128, recurrent_activation="sigmoid", return_sequences=True),
    tf.keras.layers.LSTM(128, recurrent_activation="sigmoid", return_sequences=True),
    tf.keras.layers.LSTM(128, recurrent_activation="sigmoid", return_sequences=True),
    tf.keras.layers.LSTM(128, recurrent_activation="sigmoid", return_sequences=True),
    tf.keras.layers.LSTM(128, recurrent_activation="sigmoid", return_sequences=True),
    tf.keras.layers.LSTM(128, recurrent_activation="sigmoid", return_sequences=True),
    tf.keras.layers.LSTM(128, recurrent_activation="sigmoid", return_sequences=False),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=num_target_features)
])

In [ ]:
# Compile and train this model
history_better_lstm_baseline, better_lstm_model_baseline = compile_and_fit(better_lstm_model_baseline)

In [ ]:
"""
# Evaluate the model.
test_loss, test_acc = lstm_model_baseline1.evaluate(test_ds, verbose=0)
print('Test accuracy: %.3f' % test_acc)
print('Test loss: %.4f' % test_loss)

# Plot the training curves
pd.DataFrame(history_lstm_baseline.history).plot(figsize=(8,5))
plt.show()



# Plotting the predictions of temperatures
for x, y in val_ds.take(3):
    show_plot(
        [x[0][:, 1].numpy(), y[0].numpy(), lstm_model_baseline1.predict(x)[0]],
        1,
        "Single Step Prediction",
    )
"""

In [ ]:
# Better GRU model(s): definition, compile and run, evaluate, and compare!
better_gru_model_baseline = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, gru_units]
    tf.keras.layers.GRU(128, recurrent_activation="sigmoid", return_sequences=True),
    tf.keras.layers.GRU(128, recurrent_activation="sigmoid", return_sequences=True),
    tf.keras.layers.GRU(128, recurrent_activation="sigmoid", return_sequences=True),
    tf.keras.layers.GRU(128, recurrent_activation="sigmoid", return_sequences=True),
    tf.keras.layers.GRU(128, recurrent_activation="sigmoid", return_sequences=True),
    tf.keras.layers.GRU(128, recurrent_activation="sigmoid", return_sequences=True),
    tf.keras.layers.GRU(128, recurrent_activation="sigmoid", return_sequences=False),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=num_target_features)
])

In [ ]:
# Compile and train this model
history_better_gru_model_baseline, better_gru_model_baseline = compile_and_fit(better_gru_model_baseline)

In [ ]:
# Initializing a Keras tuner based on random search for the model
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5)

In [ ]:
# Starting the search for the optimum hyperparameters for the model
#tuner.search(x_train, y_train, epochs=10, validation_data=(x_val, y_val))
#tuner.search(train_ds, to_categorical(train_labels), epochs=10, validation_data=(test_ds, to_categorical(test_labels)))
tuner.search(train_ds, validation_data=val_ds, epochs=10)

In [ ]:
best_model = tuner.get_best_models()[0]
best_model.build()
best_model.summary()

In [ ]:
build_model(keras_tuner.HyperParameters())

In [ ]:
"""
Plotting the data
"""
# Evaluate the model.
test_loss_base_model, test_acc_base_model = base_model.evaluate(test_ds, verbose=0)
print('Test accuracy base_model: %.3f' % test_acc_base_model)
print('Test loss base_model: %.4f' % test_loss_base_model)

In [ ]:
# Plot the training curves
pd.DataFrame(history_base_model.history).plot(figsize=(8,5))
plt.show()

In [ ]:
# Plotting the predictions of temperatures from lstm_model_baseline
for x, y in val_ds.take(3):
    show_plot(
        [x[0][:, 1].numpy(), y[0].numpy(), base_model.predict(x)[0]],
        1,
        "Single Step Prediction of base_model",
    )

In [ ]:
# Evaluate the model.
test_loss_lstm_model_baseline, test_acc_lstm_model_baseline = lstm_model_baseline.evaluate(test_ds, verbose=0)
print('Test accuracy lstm_model_baseline: %.3f' % test_acc_lstm_model_baseline)
print('Test loss lstm_model_baseline: %.4f' % test_loss_lstm_model_baseline)

In [ ]:
# Plot the training curves
pd.DataFrame(history_lstm_model_baseline.history).plot(figsize=(8,5))
plt.show()

In [ ]:
# Plotting the predictions of temperatures from lstm_model_baseline
for x, y in val_ds.take(3):
    show_plot(
        [x[0][:, 1].numpy(), y[0].numpy(), lstm_model_baseline.predict(x)[0]],
        1,
        "Single Step Prediction lstm_model_baseline",
    )

In [ ]:
# Evaluate the model.
test_loss_better_lstm_model_baseline, test_acc_better_lstm_model_baseline = better_lstm_model_baseline.evaluate(test_ds, verbose=0)
print('Test accuracy test_acc_better_lstm_model_baseline: %.3f' % test_acc_better_lstm_model_baseline)
print('Test loss test_acc_better_lstm_model_baseline: %.4f' % test_loss_better_lstm_model_baseline)

In [ ]:
# Plot the training curves
pd.DataFrame(history_better_lstm_model_baseline.history).plot(figsize=(8,5))
plt.show()

In [ ]:
# Plotting the predictions of temperatures
for x, y in val_ds.take(3):
    show_plot(
        [x[0][:, 1].numpy(), y[0].numpy(), better_lstm_model_baseline.predict(x)[0]],
        1,
        "Single Step Prediction test_acc_better_lstm_model_baseline",
    )

In [ ]:
# Evaluate the model.
test_loss_better_gru_model_baseline, test_acc_better_gru_model_baseline = better_gru_model_baseline.evaluate(test_ds, verbose=0)
print('Test accuracy test_acc_better_gru_model_baseline: %.3f' % test_acc_better_gru_model_baseline)
print('Test loss test_acc_better_gru_model_baseline: %.4f' % test_loss_better_gru_model_baseline)

In [ ]:
# Plot the training curves
pd.DataFrame(history_better_gru_baseline.history).plot(figsize=(8,5))
plt.show()

In [ ]:
# Plotting the predictions of temperatures
for x, y in val_ds.take(3):
    show_plot(
        [x[0][:, 1].numpy(), y[0].numpy(), better_gru_model_baseline.predict(x)[0]],
        1,
        "Single Step Prediction test_acc_better_gru_model_baseline",
    )

In [ ]:
"""
Evaluation of the models
"""
# Comparing the trained model with the baseline model statistically
# https://www.tensorflow.org/api_docs/python/tf/keras/losses/MeanAbsoluteError
base_predictions = base_model.predict(eval_ds)
trained_predictions_lstm = lstm_model_baseline.predict(eval_ds)
better_trained_predictions_lstm = better_lstm_model_baseline.predict(eval_ds)
better_trained_predictions_gru = better_gru_model_baseline.predict(eval_ds)

In [ ]:
base_mae = tf.keras.losses.MeanAbsoluteError(test_targets, base_predictions)
base_mse = tf.keras.losses.MeanAbsoluteError(test_targets, base_predictions)

In [ ]:
trained_mae_lstm = tf.keras.losses.MeanAbsoluteError(test_targets, trained_predictions_lstm)
trained_mse_lstm = tf.keras.losses.MeanAbsoluteError(test_targets, trained_predictions_lstm)

In [ ]:
better_trained_mae_lstm = tf.keras.losses.MeanAbsoluteError(test_targets, better_trained_predictions_lstm)
better_trained_mse_lstm = tf.keras.losses.MeanAbsoluteError(test_targets, better_trained_predictions_lstm)

In [ ]:
better_trained_mae_gru = tf.keras.losses.MeanAbsoluteError(test_targets, better_trained_predictions_gru)
better_trained_mse_gru = tf.keras.losses.MeanAbsoluteError(test_targets, better_trained_predictions_gru)

In [ ]:
"""
Predictions of the models
"""
# Making the predictions for the test dataset
predictions_base_model = base_model.predict(test_ds)

In [ ]:
# Extracting the predictions for the desired time steps (25 and 48 hours ahead)
predictions_base_model_25h = predictions_base_model[:, 25, lci['T (degC)']]
predictions_base_model_48h = predictions_base_model[:, 48, lci['T (degC)']]

In [ ]:
# Printing the predictions of the temperature
print("predictions_base_model 25 hours ahead:", predictions_base_model_25h)
print("predictions_base_model 48 hours ahead:", predictions_base_model_48h)

In [ ]:
# Making the predictions for the test dataset
predictions_lstm_model_baseline = lstm_model_baseline.predict(test_ds)

In [ ]:
# Extracting the predictions for the desired time steps (25 and 48 hours ahead)
predictions_lstm_model_baseline_25h = predictions_lstm_model_baseline[:, 25, lci['T (degC)']]
predictions_lstm_model_baseline_48h = predictions_lstm_model_baseline[:, 48, lci['T (degC)']]

In [ ]:
# Printing the predictions of the temperature
print("predictions_lstm_model_baseline 25 hours ahead:", predictions_lstm_model_baseline_25h)
print("predictions_lstm_model_baseline 48 hours ahead:", predictions_lstm_model_baseline_48h)

In [ ]:
# Making the predictions for the test dataset
predictions_better_lstm_model_baseline = better_lstm_model_baseline.predict(test_ds)

In [ ]:
# Extracting the predictions for the desired time steps (25 and 48 hours ahead)
predictions_better_lstm_model_baseline_25h = predictions_better_lstm_model_baseline[:, 25, lci['T (degC)']]
predictions_better_lstm_model_baseline_48h = predictions_better_lstm_model_baseline[:, 48, lci['T (degC)']]

In [ ]:
# Printing the predictions of the temperature
print("predictions_better_lstm_model_baseline 25 hours ahead:", predictions_better_lstm_model_baseline_25h)
print("predictions_better_lstm_model_baseline 48 hours ahead:", predictions_better_lstm_model_baseline_48h)

In [ ]:
# Making the predictions for the test dataset
predictions_better_gru_model_baseline = better_gru_model_baseline.predict(test_ds)

In [ ]:
# Extracting the predictions for the desired time steps (25 and 48 hours ahead)
predictions_better_gru_model_baseline_25h = predictions_better_gru_model_baseline[:, 25, lci['T (degC)']]
predictions_better_gru_model_baseline_48h = predictions_better_gru_model_baseline[:, 48, lci['T (degC)']]

In [ ]:
# Printing the predictions of the temperature
print("predictions_better_gru_model_baseline 25 hours ahead:", predictions_better_gru_model_baseline_25h)
print("predictions_better_gru_model_baseline 48 hours ahead:", predictions_better_gru_model_baseline_48h)